In [1]:
from google.colab import files
uploaded = files.upload()

Saving fixed_ks-projects-201612.csv to fixed_ks-projects-201612.csv


In [59]:
import pandas as pd

df = pd.read_csv("fixed_ks-projects-201612.csv")

# If you want to exclude the last 4 columns:
df = df.iloc[:, :-4]

df.head()

<ipython-input-59-41870e00461b>:3: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("fixed_ks-projects-201612.csv")


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375


In [60]:
#rename of name
df = df.rename(columns={'ID ': 'ID', 'name ':'name', 'category ':'category', 'main_category ':'main_category',
                       'currency ':'currency', 'deadline ': 'deadline', 'goal ':'goal', 'launched ': 'launched', 'pledged ':'pledged',
                       'state ': 'state', 'backers ': 'backers', 'country ': 'country', 'usd pledged ':'usd_pledged'})

In [61]:
df['ID']=df['ID'].astype(object)

# Convert to numeric, setting errors='coerce' to convert non-numeric values to NaN
df['goal'] = pd.to_numeric(df['goal'], errors='coerce')
print('tyep of goal:', df['goal'].dtype)  # Should now be float64

df['pledged']= pd.to_numeric(df['pledged'], errors='coerce')
print('tyep of pledged:', df['pledged'].dtype)

df['backers']=pd.to_numeric(df['backers'], errors='coerce')
print('type of backers:', df['backers'].dtype)

df['usd_pledged']=pd.to_numeric(df['usd_pledged'], errors='coerce')
print('type of usd pledged:', df['usd_pledged'].dtype)

pd.options.display.float_format = '{:.0f}'.format

tyep of goal: float64
tyep of pledged: float64
type of backers: float64
type of usd pledged: float64


In [62]:
df['deadline'] = pd.to_datetime(df['deadline'], errors='coerce')
df['deadline'] = df['deadline'].dt.floor('ns')
df['formatted'] = df['deadline'].dt.strftime('%Y-%m-%d')

df['launched'] = pd.to_datetime(df['launched'], errors='coerce')
df['launched'] = df['launched'].dt.floor('ns')
df['formatted'] = df['launched'].dt.strftime('%Y-%m-%d')

print(df['deadline'].dtype)
print(df['launched'].dtype)

datetime64[ns]
datetime64[ns]


In [63]:
# Define the list of states and main categories to keep
state_to_keep = ['failed', 'successful', 'canceled', 'live', 'suspended']
valid_main_cate = ['Film & Video', 'Music', 'Publishing', 'Games', 'Technology',
                   'Art', 'Design', 'Food', 'Fashion', 'Theater', 'Photography',
                   'Comics', 'Crafts', 'Journalism', 'Dance']

# Filter the DataFrame
df = df[(df['state'].isin(state_to_keep)) &
                (df['main_category'].isin(valid_main_cate))]

print("Shape of the cleaned DataFrame:", df.shape)

Shape of the cleaned DataFrame: (319328, 14)


In [64]:
df=df.sample(n=2000)

Preprocessing

In [65]:
target = df['state']
df = df.drop('state', axis=1)

# Perform your operations on target as intended
target = target.replace({'failed': 0, 'canceled': 0, 'live': 0, 'suspended': 0, 'successful': 1})
target = target.astype(int)


In [66]:
# preprocessing using pineline

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

num_feas=['goal', 'pledged', 'backers', 'usd_pledged']
cat_feas=['main_category', 'currency','country']

num_transformer=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

cat_transformer=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

from sklearn.compose import ColumnTransformer
preprocessor=ColumnTransformer(transformers=[
    ('num', num_transformer, num_feas),
    ('cat', cat_transformer, cat_feas)])

df= preprocessor.fit_transform(df)

In [67]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df,target,test_size=0.2, random_state=42)


In [68]:
# extreme value

from sklearn.base import BaseEstimator, TransformerMixin

class IQR(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        # Convert X to DataFrame if it's a sparse matrix
        if isinstance(X, pd.DataFrame):
            self.numeric_cols = X.select_dtypes(include=['number']).columns
        else:
            X = pd.DataFrame(X.toarray())  # Convert sparse matrix to dense DataFrame
            self.numeric_cols = X.columns  # All columns are numeric in this case

        self.q1 = X[self.numeric_cols].quantile(0.25)
        self.q3 = X[self.numeric_cols].quantile(0.75)
        self.iqr = self.q3 - self.q1
        return self

    def transform(self, X, y=None):
        # Convert X to DataFrame if it's a sparse matrix
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X.toarray())  # Convert sparse matrix to dense DataFrame

        # Calculate the lower and upper bounds for numeric columns only
        lower_bound = self.q1 - 1.5 * self.iqr
        upper_bound = self.q3 + 1.5 * self.iqr

        # Apply clipping only to numeric columns
        X[self.numeric_cols] = X[self.numeric_cols].clip(lower=lower_bound, upper=upper_bound, axis=1)
        return X

iqr = IQR()
X_train = iqr.fit_transform(X_train)
X_test = iqr.transform(X_test)

print("Columns in X_train:", X_train.columns)
print("Columns in X_test:", X_test.columns)

Columns in X_train: RangeIndex(start=0, stop=48, step=1)
Columns in X_test: RangeIndex(start=0, stop=48, step=1)


In [ ]:
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

KeyboardInterrupt: 

1. LogisticRegression

In [12]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy = lr.score(X_test, y_test)

print(f"Model accuracy: {accuracy:.4f}")

Model accuracy: 0.8350


In [20]:
#classification_report
print(f"Model accuracy of logistic regression: {accuracy:.4f}")

from sklearn.metrics import classification_report, confusion_matrix
report = classification_report(y_test, y_pred)
print('classification report:\n',report)

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print('confusion matrix:\n',conf_matrix)

Model accuracy of logistic regression: 0.8350
classification report:
               precision    recall  f1-score   support

           0       0.83      0.94      0.88       263
           1       0.84      0.64      0.73       137

    accuracy                           0.83       400
   macro avg       0.84      0.79      0.80       400
weighted avg       0.84      0.83      0.83       400

confusion matrix:
 [[246  17]
 [ 49  88]]


Modelling 2: SVC

In [27]:
from sklearn import svm
clf = svm.SVC(gamma=0.01, kernel='poly')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


Predicted,0
Actual,
0,263
1,137


In [30]:
accuracy = clf.score(X_test, y_test)
print(f"Model accuracy of logistic regression: {accuracy:.4f}")
print('classification report:\n',classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

Model accuracy of logistic regression: 0.6575
classification report:
               precision    recall  f1-score   support

           0       0.66      1.00      0.79       263
           1       0.00      0.00      0.00       137

    accuracy                           0.66       400
   macro avg       0.33      0.50      0.40       400
weighted avg       0.43      0.66      0.52       400

Confusion Matrix:
 [[263   0]
 [137   0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Modelling 3:Random Forest

In [69]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_jobs=-1,random_state=321)
rf.fit(X_train,y_train)

y_pred=rf.predict(X_test)
accuracy=rf.score(X_test,y_test)
print(f"Model accuracy: {accuracy:.4f}")
print('Model accuracy test:', rf.score(X_train,y_train))
#check overfitting: hyparam metrics of rf

from sklearn.metrics import classification_report, confusion_matrix
#classfication report and confusion matrix
report = classification_report(y_test, y_pred)
print('classification report:\n',report)

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n",conf_matrix)

Model accuracy: 0.9550
Model accuracy test: 0.990625
classification report:
               precision    recall  f1-score   support

           0       0.98      0.96      0.97       269
           1       0.91      0.95      0.93       131

    accuracy                           0.95       400
   macro avg       0.94      0.95      0.95       400
weighted avg       0.96      0.95      0.96       400

Confusion Matrix:
 [[257  12]
 [  6 125]]


In [70]:
from sklearn.metrics import classification_report, confusion_matrix
#classfication report and confusion matrix
report = classification_report(y_test, y_pred)
print(report)

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       269
           1       0.91      0.95      0.93       131

    accuracy                           0.95       400
   macro avg       0.94      0.95      0.95       400
weighted avg       0.96      0.95      0.96       400

[[257  12]
 [  6 125]]


In [43]:
#cross validation with GridSearchCV
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20, 30],'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'max_features': ['auto', 'sqrt', 'log2'] }
grid_rf = GridSearchCV(rf, parameters, cv=5)
grid_rf.fit(X_train, y_train)

print(pd.DataFrame.from_dict(grid_rf.cv_results_).loc[:, ['params', 'mean_test_score']])
print(f"Best parameters: {grid_rf.best_params_}")
print(f"Best cross-validation score: {grid_rf.best_score_:.4f}")

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
540 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/uti

                                                params  mean_test_score
0    {'max_depth': None, 'max_features': 'auto', 'm...              NaN
1    {'max_depth': None, 'max_features': 'auto', 'm...              NaN
2    {'max_depth': None, 'max_features': 'auto', 'm...              NaN
3    {'max_depth': None, 'max_features': 'auto', 'm...              NaN
4    {'max_depth': None, 'max_features': 'auto', 'm...              NaN
..                                                 ...              ...
319  {'max_depth': 30, 'max_features': 'log2', 'min...                1
320  {'max_depth': 30, 'max_features': 'log2', 'min...                1
321  {'max_depth': 30, 'max_features': 'log2', 'min...                1
322  {'max_depth': 30, 'max_features': 'log2', 'min...                1
323  {'max_depth': 30, 'max_features': 'log2', 'min...                1

[324 rows x 2 columns]
Best parameters: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_est

In [47]:
# Initialize the RandomForestClassifier with correct syntax
rf = RandomForestClassifier(
    max_depth=10,
    max_features='sqrt',
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=300,
    n_jobs=-1,
    random_state=321
)

# Fit the model
rf.fit(X_train, y_train)

# Predict on the test set
y_pred = rf.predict(X_test)

# Calculate the accuracy on the test set
accuracy = rf.score(X_test, y_test)
print(f"Model accuracy on the test set: {accuracy:.4f}")

# Check for overfitting by comparing accuracy on the training set
train_accuracy = rf.score(X_train, y_train)
print(f"Model accuracy on the training set: {train_accuracy:.4f}")

# Generate a classification report
report = classification_report(y_test, y_pred)
print('Classification Report:\n', report)

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)


Model accuracy on the test set: 0.9800
Model accuracy on the training set: 0.9850
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.98       263
           1       0.95      0.99      0.97       137

    accuracy                           0.98       400
   macro avg       0.97      0.98      0.98       400
weighted avg       0.98      0.98      0.98       400

Confusion Matrix:
 [[256   7]
 [  1 136]]


In [71]:
# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
biases = []
variances = []

for train_index, val_index in kf.split(X_train):
    X_train_fold = X_train.iloc[train_index]
    X_val_fold = X_train.iloc[val_index]
    y_train_fold = y_train.iloc[train_index]
    y_val_fold = y_train.iloc[val_index]

    # Train the model
    rf.fit(X_train_fold, y_train_fold)

    # Predict on validation set
    predictions = rf.predict(X_val_fold)

    # Compute bias
    bias = np.mean(predictions != y_val_fold)
    biases.append(bias)

    # Compute variance
    fold_predictions = np.zeros((len(y_val_fold), len(rf.estimators_)))
    for i, tree in enumerate(rf.estimators_):
        fold_predictions[:, i] = tree.predict(X_val_fold)

    variance = np.mean(np.var(fold_predictions, axis=1))
    variances.append(variance)

# Compute average bias and variance
average_bias = np.mean(biases)
average_variance = np.mean(variances)

print(f"Average Bias: {average_bias:.4f}")
print(f"Average Variance: {average_variance:.4f}")

# Evaluate on the test set
y_test_pred = rf.predict(X_test)
test_error = mean_squared_error(y_test, y_test_pred, squared=False)  # Root Mean Squared Error (RMSE)
print(f"Test RMSE: {test_error:.4f}")


Average Bias: 0.0356
Average Variance: 0.0444
Test RMSE: 0.2236


In [74]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Verify that y_test and y_pred have the same length
# Calculate regression metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print the metrics
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R-squared (R²): {r2:.4f}")

Mean Absolute Error (MAE): 0.0450
Mean Squared Error (MSE): 0.0450
Root Mean Squared Error (RMSE): 0.2121
R-squared (R²): 0.7957


In [75]:
# Extract True Positive, False Positive, True Negative, False Negative
tn, fp, fn, tp = conf_matrix.ravel()  # This works for binary classification

# Calculate False Positives (Type I Error) and False Negatives (Type II Error)
false_positives = fp
false_negatives = fn

print(f"False Positives (Type I Error): {false_positives}")
print(f"False Negatives (Type II Error): {false_negatives}")


False Positives (Type I Error): 12
False Negatives (Type II Error): 6
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.96      0.97       269
           1       0.91      0.95      0.93       131

    accuracy                           0.95       400
   macro avg       0.94      0.95      0.95       400
weighted avg       0.96      0.95      0.96       400



SHAP for interpreting model

In [ ]:
pip install shap

In [ ]:
import shap
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import pandas as pd

# Load the Iris dataset
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Create a SHAP explainer
explainer = shap.TreeExplainer(rf)

# Get SHAP values for the test set
shap_values = explainer.shap_values(X_test)

# Verify the type and shape of shap_values
print(type(shap_values))  # Should be a list
print(len(shap_values))   # Number of classes
for i, values in enumerate(shap_values):
    print(f"Shape of shap_values for class {i}:", values.shape)
print("Shape of X_test:", X_test.shape)


<class 'numpy.ndarray'>
45
Shape of shap_values for class 0: (4, 3)
Shape of shap_values for class 1: (4, 3)
Shape of shap_values for class 2: (4, 3)
Shape of shap_values for class 3: (4, 3)
Shape of shap_values for class 4: (4, 3)
Shape of shap_values for class 5: (4, 3)
Shape of shap_values for class 6: (4, 3)
Shape of shap_values for class 7: (4, 3)
Shape of shap_values for class 8: (4, 3)
Shape of shap_values for class 9: (4, 3)
Shape of shap_values for class 10: (4, 3)
Shape of shap_values for class 11: (4, 3)
Shape of shap_values for class 12: (4, 3)
Shape of shap_values for class 13: (4, 3)
Shape of shap_values for class 14: (4, 3)
Shape of shap_values for class 15: (4, 3)
Shape of shap_values for class 16: (4, 3)
Shape of shap_values for class 17: (4, 3)
Shape of shap_values for class 18: (4, 3)
Shape of shap_values for class 19: (4, 3)
Shape of shap_values for class 20: (4, 3)
Shape of shap_values for class 21: (4, 3)
Shape of shap_values for class 22: (4, 3)
Shape of shap_val

In [ ]:
print("Shape of X_test:", X_test.shape)
print(X_test.head())


Shape of X_test: (45, 4)
     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
73                   6                 3                  5                 1
18                   6                 4                  2                 0
118                  8                 3                  7                 2
78                   6                 3                  4                 2
76                   7                 3                  5                 1


In [ ]:
import shap
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import pandas as pd

# Load the Iris dataset
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Create a SHAP explainer
explainer = shap.TreeExplainer(rf)

# Get SHAP values for the test set
shap_values = explainer.shap_values(X_test)

# Check the type and shape of shap_values
print(type(shap_values))  # Should be a list
print(len(shap_values))   # Should match number of classes
for i, values in enumerate(shap_values):
    print(f"Shape of shap_values for class {i}:", values.shape)

# Generate the summary plot for class 0 (or another class)
shap.summary_plot(shap_values[0], X_test)

# Generate a dependence plot for a specific feature (e.g., feature index 0)
shap.dependence_plot(0, shap_values[0], X_test)

# Generate a force plot for the first test instance (e.g., index 0)
shap.force_plot(explainer.expected_value[0], shap_values[0][0], X_test.iloc[0])




<class 'numpy.ndarray'>
45
Shape of shap_values for class 0: (4, 3)
Shape of shap_values for class 1: (4, 3)
Shape of shap_values for class 2: (4, 3)
Shape of shap_values for class 3: (4, 3)
Shape of shap_values for class 4: (4, 3)
Shape of shap_values for class 5: (4, 3)
Shape of shap_values for class 6: (4, 3)
Shape of shap_values for class 7: (4, 3)
Shape of shap_values for class 8: (4, 3)
Shape of shap_values for class 9: (4, 3)
Shape of shap_values for class 10: (4, 3)
Shape of shap_values for class 11: (4, 3)
Shape of shap_values for class 12: (4, 3)
Shape of shap_values for class 13: (4, 3)
Shape of shap_values for class 14: (4, 3)
Shape of shap_values for class 15: (4, 3)
Shape of shap_values for class 16: (4, 3)
Shape of shap_values for class 17: (4, 3)
Shape of shap_values for class 18: (4, 3)
Shape of shap_values for class 19: (4, 3)
Shape of shap_values for class 20: (4, 3)
Shape of shap_values for class 21: (4, 3)
Shape of shap_values for class 22: (4, 3)
Shape of shap_val

AssertionError: The shape of the shap_values matrix does not match the shape of the provided data matrix.

Modelling 4: XGBoost

In [36]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
accuracy = xgb_model.score(X_test, y_test)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:15:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [37]:
print(f"XGBoost Model accuracy: {accuracy:.4f}")
#classfication report and confusion matrix
report = classification_report(y_test, y_pred)
print('classification report:\n',report)

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print('confusion matrix:\n',conf_matrix)

XGBoost Model accuracy: 0.9825
classification report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99       263
           1       0.96      0.99      0.97       137

    accuracy                           0.98       400
   macro avg       0.98      0.98      0.98       400
weighted avg       0.98      0.98      0.98       400

confusion matrix:
 [[257   6]
 [  1 136]]


Deep learning

In [38]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
model = Sequential()
# Input Layer
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.3))

# Hidden Layer 1
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

# Hidden Layer 2
model.add(Dense(32, activation='relu'))

# Output Layer (Binary Classification: 0 or 1 for success/failure)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.5739 - loss: 0.6669 - val_accuracy: 0.6575 - val_loss: 0.5910
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7075 - loss: 0.5832 - val_accuracy: 0.7850 - val_loss: 0.4593
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7896 - loss: 0.4719 - val_accuracy: 0.8375 - val_loss: 0.3650
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8323 - loss: 0.3993 - val_accuracy: 0.8525 - val_loss: 0.3323
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8431 - loss: 0.3645 - val_accuracy: 0.8750 - val_loss: 0.3200
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8793 - loss: 0.3160 - val_accuracy: 0.8725 - val_loss: 0.3093
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8764 - loss: 0.2992 - val_accuracy: 0.8825 - val_loss: 0.2701
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8699 - loss: 0.3005 - val_accuracy: 0.8950 - val_loss

In [39]:
print(f"Test Accuracy: {accuracy:.2f}")

report = classification_report(y_test, y_pred)
print('classification report:\n',report)

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print('confusion matrix:\n',conf_matrix)

Test Accuracy: 0.93
classification report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99       263
           1       0.96      0.99      0.97       137

    accuracy                           0.98       400
   macro avg       0.98      0.98      0.98       400
weighted avg       0.98      0.98      0.98       400

confusion matrix:
 [[257   6]
 [  1 136]]
